# RecBole Research

## Atomic files and datasets

##### Atomic files using datasets obtained with `generate_data.ipynb`

### __Inters atomic file__

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

LASTFM_RB = 'lastfm_recbole'
DATA_DIR = f'../../data/recsys_data/'

In [4]:
ratings = pd.read_csv(DATA_DIR + 'all_tracks_ratings_full.zip', sep='\t')

In [8]:
all_ratings = ratings.copy()

all_ratings.columns = ['user_id:token', 'track_id:token', 'rating:float', 'timestamp:float']
all_ratings['timestamp:float'] = all_ratings['timestamp:float'].apply(
    lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M:%S").timestamp() if x is not np.nan else np.nan)

all_ratings.to_csv(f'{DATA_DIR}/{LASTFM_RB}/{LASTFM_RB}.inter', index=False, sep='\t')
all_ratings


,user_id:token,track_id:token,rating:float,timestamp:float
0,1,35151,5.0,NaN
1,1,82497,3.3,NaN
2,1,97967,4.2,NaN
3,1,105492,3.5,NaN
4,1,124021,3.9,NaN
...,...,...,...,...
2200756,52829,711172,4.2,NaN
2200757,52829,745290,1.0,1.668483e+09
2200758,52829,776707,1.0,1.668444e+09
2200759,52829,781335,1.0,1.668444e+09


### __Users atomic file__

In [4]:
all_users = ratings['user_id'].drop_duplicates().rename('user_id:token')
all_users.to_csv(f'{DATA_DIR}/{LASTFM_RB}/{LASTFM_RB}.user', index=False, sep='\t')

all_users

0              1
20             2
60             3
119            4
148            5
           ...  
2200552    52825
2200587    52826
2200624    52827
2200663    52828
2200703    52829
Name: user_id:token, Length: 52780, dtype: int64

### __Items atomic file__

Choose columns to load by editing `config.yaml`

In [2]:
OG_COLS = ['V', 'A', 'D', 'StSc']
NORM_COLS = ['V_Norm', 'A_Norm', 'D_Norm', 'StSc_Norm']

RECBOLE_TAG_COLS = ['track_id:token', 'tags:token_seq']
RECBOLE_VAD_COLS = ['track_id:token', 'vadst:float_seq']

track_data = pd.read_csv(DATA_DIR + "track_full_data.zip", sep='\t')

track_data

,track_id,tags,name,artist_id,album_id
0,1,folk folk folk folk folk folk folk folk countr...,twentyseven,136900,227929.0
1,2,christmas christmas christmas christmas christ...,Happy Xmas (War Is Over) (2003 Mix),82136,NaN
2,3,wrong wrong wrong wrong wrong wrong wrong wrong,Don’t Shut Me Down,127107,NaN
3,4,melodicdeathmetal melodicdeathmetal melodicdea...,Tirpitz,57668,144260.0
4,5,slowcore slowcore slowcore slowcore slowcore s...,You Have Cum in Your Hair and Your Dick is Han...,117533,99037.0
...,...,...,...,...,...
815626,815627,frenchpop frenchpop frenchpop frenchpop french...,D-1,71909,144316.0
815627,815628,hiphop hiphop hiphop hiphop hiphop hiphop hiph...,Now or Never,125328,NaN
815628,815629,hiphop hiphop hiphop hiphop hiphop hiphop hiph...,Los Zurdos Mueren Antes,142059,NaN
815629,815630,indie indie indie indie indie indie indie indie,Oh Mama,99271,92757.0


#### Load all Data with NaNs

In [4]:
track_all = track_data[['track_id', 'tags', 'artist_id', 'album_id']]
track_all = track_all.rename(lambda x: x + ':token', axis=1).rename(columns={'tags:token': RECBOLE_TAG_COLS[1]})

track_all

,track_id:token,tags:token_seq,artist_id:token,album_id:token
0,1,folk folk folk folk folk folk folk folk countr...,136900,227929.0
1,2,christmas christmas christmas christmas christ...,82136,NaN
2,3,wrong wrong wrong wrong wrong wrong wrong wrong,127107,NaN
3,4,melodicdeathmetal melodicdeathmetal melodicdea...,57668,144260.0
4,5,slowcore slowcore slowcore slowcore slowcore s...,117533,99037.0
...,...,...,...,...
815626,815627,frenchpop frenchpop frenchpop frenchpop french...,71909,144316.0
815627,815628,hiphop hiphop hiphop hiphop hiphop hiphop hiph...,125328,NaN
815628,815629,hiphop hiphop hiphop hiphop hiphop hiphop hiph...,142059,NaN
815629,815630,indie indie indie indie indie indie indie indie,99271,92757.0


In [ ]:
track_all.to_csv(f'{DATA_DIR}/{LASTFM_RB}/{LASTFM_RB}.item', index=False, sep='\t')

### __Include VAD values__

#### Option A: Original VAD + StSc

In [5]:
all_vads = pd.read_csv(DATA_DIR + 'all_tracks_vads.zip', sep='\t')

vads = all_vads[['Track'] + OG_COLS].copy()
vads['VADStSc'] = vads[OG_COLS].astype(str).apply(lambda x: ' '.join(x), axis=1)
vads.drop(columns=OG_COLS, inplace=True)
vads.columns = RECBOLE_VAD_COLS

vads

,track_id:token,vadst:float_seq
0,1,nan nan nan nan
1,2,0.6678796252023386 0.4404606082762304 0.568477...
2,3,0.587845238095238 0.4901428571428571 0.5043333...
3,4,0.6319867998279273 0.4767079600804765 0.578362...
4,5,0.6231669245647969 0.3807249516441006 0.507896...
...,...,...
815626,815627,0.6775663607732008 0.4722983474411399 0.600010...
815627,815628,0.6514011030677977 0.4618827553321515 0.575568...
815628,815629,nan nan nan nan
815629,815630,0.6084758620689655 0.4409655172413793 0.554662...


In [ ]:
vads.to_csv(f'{DATA_DIR}/{LASTFM_RB}/{LASTFM_RB}.item', index=False, sep='\t')

#### Option B: Normalized VAD + StSc

In [6]:
all_vads = pd.read_csv(DATA_DIR + 'all_tracks_vads.zip', sep='\t')

vads = all_vads[['Track'] + NORM_COLS].copy()
vads['VADStSc'] = vads[NORM_COLS].astype(str).apply(lambda x: ' '.join(x), axis=1)
vads.drop(columns=NORM_COLS, inplace=True)
vads.columns = RECBOLE_VAD_COLS

vads

,track_id:token,vadst:float_seq
0,1,nan nan nan nan
1,2,0.5715571724783821 0.4159363290920476 0.474975...
2,3,0.4619585595278849 0.4888376480452782 0.363160...
3,4,0.5224057512193457 0.4691239326199215 0.492206...
4,5,0.5103278665728133 0.3282831278710206 0.369371...
...,...,...
815626,815627,0.5848221304665535 0.4626534812049008 0.529942...
815627,815628,0.5489915824276587 0.447370147222526 0.4873355...
815628,815629,nan nan nan nan
815629,815630,0.4902100131036843 0.4166772079844157 0.450892...


In [ ]:
vads.to_csv(f'{DATA_DIR}/{LASTFM_RB}/{LASTFM_RB}.item', index=False, sep='\t')

#### Merge Track Data with VAD + St

In [22]:
track_all['vadst:float_seq'] = vads['vadst:float_seq']
track_all

,track_id:token,tags:token_seq,artist_id:token,album_id:token,vadst:float_seq
0,1,folk folk folk folk folk folk folk folk countr...,136900,227929.0,nan nan nan nan
1,2,christmas christmas christmas christmas christ...,82136,NaN,0.5715571724783821 0.4159363290920476 0.474975...
2,3,wrong wrong wrong wrong wrong wrong wrong wrong,127107,NaN,0.4619585595278849 0.4888376480452782 0.363160...
3,4,melodicdeathmetal melodicdeathmetal melodicdea...,57668,144260.0,0.5224057512193457 0.4691239326199215 0.492206...
4,5,slowcore slowcore slowcore slowcore slowcore s...,117533,99037.0,0.5103278665728133 0.3282831278710206 0.369371...
...,...,...,...,...,...
815626,815627,frenchpop frenchpop frenchpop frenchpop french...,71909,144316.0,0.5848221304665535 0.4626534812049008 0.529942...
815627,815628,hiphop hiphop hiphop hiphop hiphop hiphop hiph...,125328,NaN,0.5489915824276587 0.447370147222526 0.4873355...
815628,815629,hiphop hiphop hiphop hiphop hiphop hiphop hiph...,142059,NaN,nan nan nan nan
815629,815630,indie indie indie indie indie indie indie indie,99271,92757.0,0.4902100131036843 0.4166772079844157 0.450892...


In [ ]:
track_all.to_csv(f'{DATA_DIR}/{LASTFM_RB}/{LASTFM_RB}.item', index=False, sep='\t')

#### __Optional__: Drop NaN values (row-wise)

In [21]:
track_all_no_nan = track_all.dropna(axis=0).copy()

def replace_nan_vadst(x):
    if x == 'nan nan nan nan':
        return '0.0 0.0 0.0 0.0'
    return x

track_all_no_nan['vadst:float_seq'] = track_all_no_nan['vadst:float_seq'].apply(replace_nan_vadst)
track_all_no_nan

,track_id:token,tags:token_seq,artist_id:token,album_id:token,vadst:float_seq
0,1,folk folk folk folk folk folk folk folk countr...,136900,227929.0,0.0 0.0 0.0 0.0
3,4,melodicdeathmetal melodicdeathmetal melodicdea...,57668,144260.0,0.5224057512193457 0.4691239326199215 0.492206...
4,5,slowcore slowcore slowcore slowcore slowcore s...,117533,99037.0,0.5103278665728133 0.3282831278710206 0.369371...
5,6,broadwayobc broadwayobc broadwayobc broadwayob...,95498,383880.0,0.438682558225717 0.5129376526915309 0.4168159...
6,7,melodicdeathmetal melodicdeathmetal melodicdea...,38330,249562.0,0.4468730540627671 0.4836855228373376 0.434426...
...,...,...,...,...,...
815623,815624,poppunk poppunk poppunk poppunk poppunk poppun...,29854,222810.0,0.0 0.0 0.0 0.0
815624,815625,canadian canadian canadian canadian canadian c...,95517,262444.0,0.5539297458930935 0.4605652517970017 0.524654...
815625,815626,rock rock rock rock rock rock rock rock female...,30287,12976.0,0.5831128626268791 0.4615351340409633 0.512471...
815626,815627,frenchpop frenchpop frenchpop frenchpop french...,71909,144316.0,0.5848221304665535 0.4626534812049008 0.529942...


In [ ]:
track_all_no_nan.to_csv(f'{DATA_DIR}/{LASTFM_RB}/{LASTFM_RB}.item', index=False, sep='\t')